In [68]:
import glob
from lxml import etree
from bs4 import BeautifulSoup
import re
import requests
import csv
from collections import Counter

In [69]:
products = glob.glob('products/*.html')

In [111]:
cgid = re.compile(r'CGR[A-Z0-9]+')
allproducts = []

for page in products:
    name = page.split('/')[-1]
    with open(page, 'rb') as fin:
        soup = BeautifulSoup(fin.read(), "html")
#         print(soup.find_all("main")[0].prettify())
        tree = etree.fromstring(soup.find_all("main")[0].prettify())
        description = tree.xpath('.//div[@itemprop = "description"]/p/text()')[0].strip()
        name = tree.xpath('.//*[@itemprop = "name"]/text()')[0].strip()
        url = tree.xpath('.//meta[@itemprop = "url"]/@content')[0]
        image = "https://" + tree.xpath('.//img[@id = "ProductPhotoImg"]/@src')[0].strip('//')
        patternid = description.split(' - ')[0]
        data = [patternid]
#         print(image)
        rulers = re.findall(cgid, description)
        if rulers:
            for r in rulers:
                data = [patternid, name, r, description, url]
                allproducts.append(data)
        else:
            data = [patternid, name, 'NONE', description, url]
            allproducts.append(data)

In [112]:
allrulers = [r[2] for r in allproducts]

In [170]:
counted_rulers = dict(Counter(allrulers))

ruler_products = {k: [] for k in counted_rulers}
# print(counted_rulers)

for ruler in ruler_products:
    url = "https://www.creativegridsusa.com/products/" + ruler
    r = requests.get(url)
    payload = r.content
#     print(payload)
    with open("rulers/" + ruler + '.html', 'wb') as fout:
        fout.write(payload.replace(b' ---', b' --').replace(b'--- ', b'-- '))
    r.close()

In [171]:
for product in allproducts:
    ruler_used = product[2]
    ruler_products[ruler_used].append(product)

In [172]:
rulernames = {k: [] for k in counted_rulers}

for rulername in rulernames:
    rulerpage = "rulers/" + rulername + '.html'
    with open(rulerpage, 'rb') as fin:
        soup = BeautifulSoup(fin.read(), "html")
#         print(soup.find_all("body")[0].prettify())
        name = soup.find_all("h1")
        if name:
            name = name[0].text.replace('Creative Grids ', '')
        else:
            name = "Unavailable"
        rulernames[rulername] = name

In [188]:
text = "Here is a sorted count of all the rulers used:\n"

text += """All data here are copyright Cut Loose Press and Creative Grids.
        I am not committing the source code for harvesting the pages or the pages themselves.
        I am including the code that extracts the information from the source pages.
        "NONE" means that no creative grids ruler was mentioned and "unavailable" means that the link to the ruler is dead.
        \n\nNavigate to ruler_groups to see individual pages for Cut Loose Press patterns that use those specific Creative Grid rulers.\n\n"""

sortedcounts = sorted(counted_rulers.items(), key=lambda i: i[1], reverse=True)

for r, count in sortedcounts:
    text += "\n* " + str(count) + " patterns: " + r + ": (" + rulernames[r].strip() + ")"
    text += "\n\t* https://www.creativegridsusa.com/products/" + r
    
text += "\n\n"

with open("readme.md", 'wt') as fout:
    fout.write(text)

for ruler, patterns in ruler_products.items():
    text = ""
    with open("ruler_groups/" + ruler + ".md", 'wt') as fout:
        text += "# " + rulernames[r].strip()
        text += "\n* https://www.creativegridsusa.com/products/" + r + "\n"
        text += "\n## " + ruler + '\n\nThe following patterns use this ruler, (' + str(len(patterns)) + " patterns)\n"
        for p in patterns:
            text += "\n* " + p[0] + '--' + p[1] + '\n\t* ' + p[3] + '\n\t* ' + p[4] + "\n\n"
        fout.write(text)
        


In [175]:
short = ""

for r, count in sortedcounts:
    short += "\n* " + str(count) + " patterns: " + r + ": (" + rulernames[r].strip() + ")"

In [177]:
print(short)


* 59 patterns: NONE: (Unavailable)
* 16 patterns: CGRJAWMN4: (4in Log Cabin Trim Tool Quilt Ruler)
* 13 patterns: CGRJAW1: (Log Cabin Trim Tool for 8in Finished Blocks Quilt Ruler)
* 13 patterns: CGRJAW6MINI: (4in Curvy Log Cabin Trim Tool)
* 11 patterns: CGRDH2: (Strippy Stars Tool 5in x 7-1/2in Quilt Ruler)
* 11 patterns: CGRJAW5: (Curvy Log Cabin Trim Tool 8in Finished Blocks)
* 11 patterns: CGRBH1: (Half-Square 4-in-1 Triangle Quilt Ruler)
* 11 patterns: CGR24: (Quilt Ruler 6-1/2in x 24-1/2in)
* 10 patterns: CGRJAW8: (Square on Square Trim Tool - 4in or 8in Finished)
* 10 patterns: CGRJAW3MINI: (Pineapple Trim Tool Mini Quilt Ruler)
* 10 patterns: CGRJAW3: (Pineapple Trim Tool for 6, 8 or 10in Finished Blocks)
* 10 patterns: CGRDH4: (Ultimate Flying Geese Tool)
* 9 patterns: CGRGE1: (Unavailable)
* 8 patterns: CGRDH1: (Cat's Cradle Tool Quilt Ruler)
* 8 patterns: CGRJAW4: (Hexagon Trim Tool Quilt Ruler)
* 8 patterns: CGRT60: (60 Degree Triangle 8-1/2in Quilt Ruler)
* 8 patterns: C